In [1]:
import sys

from morpheus.project.types.geometry.MultiPolygon import MultiPolygon

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid, ActiveCells
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()

grid = Grid.cartesian_from_polygon(
  polygon=polygon,
  n_rows=2000,
  n_cols=2000,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')


## Performance of ActiveCells Calculation with a Point

In [3]:
%%time
from morpheus.project.types.geometry import Point

point_within = Point(
  type='Point',
  coordinates=(13.923, 50.965)
)

ac = ActiveCells.from_point(point=point_within, grid=grid)
len(ac)




CPU times: user 61 ms, sys: 87.7 ms, total: 149 ms
Wall time: 28.9 ms


1

## Performance of ActiveCells Calculation with a LineString 

In [4]:
from morpheus.project.types.geometry import LineString

linestring_within = LineString(
  type='LineString',
  coordinates=[
    (13.923, 50.965),
    (13.924, 50.965),
  ]
)

In [5]:
%%time
from morpheus.project.types.discretization.spatial import ActiveCells

ac = ActiveCells.from_linestring(linestring=linestring_within, grid=grid)
len(ac)

CPU times: user 442 ms, sys: 3.65 ms, total: 446 ms
Wall time: 306 ms


590

## Performance of ActiveCells Calculation with a Polygon

In [6]:
from morpheus.project.types.geometry import Polygon

polygon_within = Polygon(
  type='Polygon',
  coordinates=[[
    (13.923, 50.965),
    (13.924, 50.9655),
    (13.925, 50.965),
    (13.925, 50.966),

    (13.923, 50.966),
    (13.923, 50.965),
  ]]
)

In [7]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m


In [8]:
%%time
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)
len(active_cells)

[(720, 345)] [(722, 345)]
[(720, 346)] [(724, 346)]
[(720, 347)] [(726, 347)]
[(719, 348)] [(728, 348)]
[(719, 349)] [(730, 349)]
[(719, 350)] [(732, 350)]
[(719, 351)] [(734, 351)]
[(718, 352)] [(736, 352)]
[(718, 353)] [(738, 353)]
[(718, 354)] [(740, 354)]
[(718, 355)] [(742, 355)]
[(717, 356)] [(744, 356)]
[(717, 357)] [(746, 357)]
[(717, 358)] [(748, 358)]
[(717, 359)] [(750, 359)]
[(716, 360)] [(752, 360)]
[(716, 361)] [(754, 361)]
[(716, 362)] [(756, 362)]
[(715, 363)] [(758, 363)]
[(715, 364)] [(760, 364)]
[(715, 365)] [(762, 365)]
[(715, 366)] [(764, 366)]
[(714, 367)] [(766, 367)]
[(714, 368)] [(768, 368)]
[(714, 369)] [(770, 369)]
[(714, 370)] [(772, 370)]
[(713, 371)] [(774, 371)]
[(713, 372)] [(776, 372)]
[(713, 373)] [(778, 373)]
[(713, 374)] [(780, 374)]
[(712, 375)] [(782, 375)]
[(712, 376)] [(784, 376)]
[(712, 377)] [(786, 377)]
[(712, 378)] [(788, 378)]
[(711, 379)] [(790, 379)]
[(711, 380)] [(792, 380)]
[(711, 381)] [(794, 381)]
[(710, 382)] [(796, 382)]
[(710, 383)]

[(475, 1271)] [(1808, 1271)]
[(475, 1272)] [(1808, 1272)]
[(475, 1273)] [(1808, 1273)]
[(475, 1274)] [(1807, 1274)]
[(474, 1275)] [(1807, 1275)]
[(474, 1276)] [(1807, 1276)]
[(474, 1277)] [(1807, 1277)]
[(473, 1278)] [(1806, 1278)]
[(473, 1279)] [(1806, 1279)]
[(473, 1280)] [(1806, 1280)]
[(473, 1281)] [(1806, 1281)]
[(472, 1282)] [(1805, 1282)]
[(472, 1283)] [(1137, 1283)]
[(1139, 1283)] [(1805, 1283)]
[(472, 1284)] [(1135, 1284)]
[(1140, 1284)] [(1805, 1284)]
[(472, 1285)] [(1132, 1285)]
[(1140, 1285)] [(1804, 1285)]
[(471, 1286)] [(1130, 1286)]
[(1141, 1286)] [(1804, 1286)]
[(471, 1287)] [(1128, 1287)]
[(1141, 1287)] [(1804, 1287)]
[(471, 1288)] [(1126, 1288)]
[(1141, 1288)] [(1804, 1288)]
[(471, 1289)] [(1124, 1289)]
[(1142, 1289)] [(1803, 1289)]
[(470, 1290)] [(1121, 1290)]
[(1142, 1290)] [(1803, 1290)]
[(470, 1291)] [(1119, 1291)]
[(1143, 1291)] [(1803, 1291)]
[(470, 1292)] [(1117, 1292)]
[(1143, 1292)] [(1803, 1292)]
[(469, 1293)] [(1115, 1293)]
[(1144, 1293)] [(1802, 1293)]
[(4

1270195

## Performance of ActiveCells Calculation with a MultiPolygon

In [9]:
%%time
multi_polygon_within = MultiPolygon(
  type='MultiPolygon',
  coordinates=[
    [[
      (13.923, 50.965),
      (13.924, 50.9655),
      (13.925, 50.965),
      (13.925, 50.966),
      (13.923, 50.966),
      (13.923, 50.965),
    ]],
    [[
      (13.9235, 50.9655),
      (13.924, 50.966),
      (13.9245, 50.9655),
      (13.9235, 50.9655),
    ]]
  ]
)

active_cells = ActiveCells.from_multipolygon(multipolygon=multi_polygon_within, grid=grid)
len(active_cells)

[(720, 345)] [(722, 345)]
[(720, 346)] [(724, 346)]
[(720, 347)] [(726, 347)]
[(719, 348)] [(728, 348)]
[(719, 349)] [(730, 349)]
[(719, 350)] [(732, 350)]
[(719, 351)] [(734, 351)]
[(718, 352)] [(736, 352)]
[(718, 353)] [(738, 353)]
[(718, 354)] [(740, 354)]
[(718, 355)] [(742, 355)]
[(717, 356)] [(744, 356)]
[(717, 357)] [(746, 357)]
[(717, 358)] [(748, 358)]
[(717, 359)] [(750, 359)]
[(716, 360)] [(752, 360)]
[(716, 361)] [(754, 361)]
[(716, 362)] [(756, 362)]
[(715, 363)] [(758, 363)]
[(715, 364)] [(760, 364)]
[(715, 365)] [(762, 365)]
[(715, 366)] [(764, 366)]
[(714, 367)] [(766, 367)]
[(714, 368)] [(768, 368)]
[(714, 369)] [(770, 369)]
[(714, 370)] [(772, 370)]
[(713, 371)] [(774, 371)]
[(713, 372)] [(776, 372)]
[(713, 373)] [(778, 373)]
[(713, 374)] [(780, 374)]
[(712, 375)] [(782, 375)]
[(712, 376)] [(784, 376)]
[(712, 377)] [(786, 377)]
[(712, 378)] [(788, 378)]
[(711, 379)] [(790, 379)]
[(711, 380)] [(792, 380)]
[(711, 381)] [(794, 381)]
[(710, 382)] [(796, 382)]
[(710, 383)]

[(561, 946)] [(1894, 946)]
[(561, 947)] [(1894, 947)]
[(561, 948)] [(1894, 948)]
[(560, 949)] [(1893, 949)]
[(560, 950)] [(1893, 950)]
[(560, 951)] [(1893, 951)]
[(560, 952)] [(1893, 952)]
[(559, 953)] [(1892, 953)]
[(559, 954)] [(1892, 954)]
[(559, 955)] [(1892, 955)]
[(559, 956)] [(1892, 956)]
[(558, 957)] [(1891, 957)]
[(558, 958)] [(1891, 958)]
[(558, 959)] [(1891, 959)]
[(558, 960)] [(1890, 960)]
[(557, 961)] [(1890, 961)]
[(557, 962)] [(1890, 962)]
[(557, 963)] [(1890, 963)]
[(557, 964)] [(1889, 964)]
[(556, 965)] [(1889, 965)]
[(556, 966)] [(1889, 966)]
[(556, 967)] [(1889, 967)]
[(555, 968)] [(1888, 968)]
[(555, 969)] [(1888, 969)]
[(555, 970)] [(1888, 970)]
[(555, 971)] [(1888, 971)]
[(554, 972)] [(1887, 972)]
[(554, 973)] [(1887, 973)]
[(554, 974)] [(1887, 974)]
[(554, 975)] [(1886, 975)]
[(553, 976)] [(1886, 976)]
[(553, 977)] [(1886, 977)]
[(553, 978)] [(1886, 978)]
[(553, 979)] [(1885, 979)]
[(552, 980)] [(1885, 980)]
[(552, 981)] [(1885, 981)]
[(552, 982)] [(1885, 982)]
[

[(1173, 1359)] [(1785, 1359)]
[(452, 1360)] [(969, 1360)]
[(1174, 1360)] [(1785, 1360)]
[(452, 1361)] [(967, 1361)]
[(1174, 1361)] [(1784, 1361)]
[(451, 1362)] [(965, 1362)]
[(1175, 1362)] [(1784, 1362)]
[(451, 1363)] [(962, 1363)]
[(1175, 1363)] [(1784, 1363)]
[(451, 1364)] [(960, 1364)]
[(1175, 1364)] [(1784, 1364)]
[(450, 1365)] [(958, 1365)]
[(1176, 1365)] [(1783, 1365)]
[(450, 1366)] [(956, 1366)]
[(1176, 1366)] [(1783, 1366)]
[(450, 1367)] [(954, 1367)]
[(1177, 1367)] [(1783, 1367)]
[(450, 1368)] [(952, 1368)]
[(1177, 1368)] [(1783, 1368)]
[(449, 1369)] [(949, 1369)]
[(1178, 1369)] [(1782, 1369)]
[(449, 1370)] [(947, 1370)]
[(1178, 1370)] [(1782, 1370)]
[(449, 1371)] [(945, 1371)]
[(1179, 1371)] [(1782, 1371)]
[(449, 1372)] [(943, 1372)]
[(1179, 1372)] [(1781, 1372)]
[(448, 1373)] [(941, 1373)]
[(1179, 1373)] [(1781, 1373)]
[(448, 1374)] [(938, 1374)]
[(1180, 1374)] [(1781, 1374)]
[(448, 1375)] [(936, 1375)]
[(1180, 1375)] [(1781, 1375)]
[(448, 1376)] [(934, 1376)]
[(1181, 1376)]

[(1308, 640)] [(1309, 640)]
[(1307, 641)] [(1309, 641)]
[(1306, 642)] [(1309, 642)]
[(1305, 643)] [(1309, 643)]
[(1304, 644)] [(1310, 644)]
[(1303, 645)] [(1310, 645)]
[(1302, 646)] [(1310, 646)]
[(1301, 647)] [(1310, 647)]
[(1301, 648)] [(1310, 648)]
[(1300, 649)] [(1310, 649)]
[(1299, 650)] [(1311, 650)]
[(1298, 651)] [(1311, 651)]
[(1297, 652)] [(1311, 652)]
[(1296, 653)] [(1311, 653)]
[(1295, 654)] [(1311, 654)]
[(1294, 655)] [(1311, 655)]
[(1293, 656)] [(1312, 656)]
[(1292, 657)] [(1312, 657)]
[(1291, 658)] [(1312, 658)]
[(1290, 659)] [(1312, 659)]
[(1289, 660)] [(1312, 660)]
[(1288, 661)] [(1312, 661)]
[(1287, 662)] [(1312, 662)]
[(1286, 663)] [(1313, 663)]
[(1286, 664)] [(1313, 664)]
[(1285, 665)] [(1313, 665)]
[(1284, 666)] [(1313, 666)]
[(1283, 667)] [(1313, 667)]
[(1282, 668)] [(1313, 668)]
[(1281, 669)] [(1314, 669)]
[(1280, 670)] [(1314, 670)]
[(1279, 671)] [(1314, 671)]
[(1278, 672)] [(1314, 672)]
[(1277, 673)] [(1314, 673)]
[(1276, 674)] [(1314, 674)]
[(1275, 675)] [(1315

1270195

## Time to generate the outline of the active cells

In [10]:
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)
len(active_cells)

[(720, 345)] [(722, 345)]
[(720, 346)] [(724, 346)]
[(720, 347)] [(726, 347)]
[(719, 348)] [(728, 348)]
[(719, 349)] [(730, 349)]
[(719, 350)] [(732, 350)]
[(719, 351)] [(734, 351)]
[(718, 352)] [(736, 352)]
[(718, 353)] [(738, 353)]
[(718, 354)] [(740, 354)]
[(718, 355)] [(742, 355)]
[(717, 356)] [(744, 356)]
[(717, 357)] [(746, 357)]
[(717, 358)] [(748, 358)]
[(717, 359)] [(750, 359)]
[(716, 360)] [(752, 360)]
[(716, 361)] [(754, 361)]
[(716, 362)] [(756, 362)]
[(715, 363)] [(758, 363)]
[(715, 364)] [(760, 364)]
[(715, 365)] [(762, 365)]
[(715, 366)] [(764, 366)]
[(714, 367)] [(766, 367)]
[(714, 368)] [(768, 368)]
[(714, 369)] [(770, 369)]
[(714, 370)] [(772, 370)]
[(713, 371)] [(774, 371)]
[(713, 372)] [(776, 372)]
[(713, 373)] [(778, 373)]
[(713, 374)] [(780, 374)]
[(712, 375)] [(782, 375)]
[(712, 376)] [(784, 376)]
[(712, 377)] [(786, 377)]
[(712, 378)] [(788, 378)]
[(711, 379)] [(790, 379)]
[(711, 380)] [(792, 380)]
[(711, 381)] [(794, 381)]
[(710, 382)] [(796, 382)]
[(710, 383)]

[(1143, 1292)] [(1803, 1292)]
[(469, 1293)] [(1115, 1293)]
[(1144, 1293)] [(1802, 1293)]
[(469, 1294)] [(1113, 1294)]
[(1144, 1294)] [(1802, 1294)]
[(469, 1295)] [(1111, 1295)]
[(1145, 1295)] [(1802, 1295)]
[(469, 1296)] [(1108, 1296)]
[(1145, 1296)] [(1802, 1296)]
[(468, 1297)] [(1106, 1297)]
[(1146, 1297)] [(1801, 1297)]
[(468, 1298)] [(1104, 1298)]
[(1146, 1298)] [(1801, 1298)]
[(468, 1299)] [(1102, 1299)]
[(1146, 1299)] [(1801, 1299)]
[(468, 1300)] [(1100, 1300)]
[(1147, 1300)] [(1801, 1300)]
[(467, 1301)] [(1098, 1301)]
[(1147, 1301)] [(1800, 1301)]
[(467, 1302)] [(1095, 1302)]
[(1148, 1302)] [(1800, 1302)]
[(467, 1303)] [(1093, 1303)]
[(1148, 1303)] [(1800, 1303)]
[(467, 1304)] [(1091, 1304)]
[(1149, 1304)] [(1799, 1304)]
[(466, 1305)] [(1089, 1305)]
[(1149, 1305)] [(1799, 1305)]
[(466, 1306)] [(1087, 1306)]
[(1150, 1306)] [(1799, 1306)]
[(466, 1307)] [(1084, 1307)]
[(1150, 1307)] [(1799, 1307)]
[(466, 1308)] [(1082, 1308)]
[(1150, 1308)] [(1798, 1308)]
[(465, 1309)] [(1080, 1309

1270195

In [11]:
%%time
active_cells_outline = active_cells.outline_to_geojson(grid=grid)
print(active_cells_outline)

Feature(geometry=Polygon(coordinates=(((13.922998303121227, 50.965665092827535), (13.922998325350097, 50.965665099088035), (13.922998481874712, 50.965665134967004), (13.922998576546586, 50.96566516163022), (13.922998668141261, 50.96566519746465), (13.922998755772678, 50.96566524212364), (13.922998838593118, 50.96566529517518), (13.922998915801402, 50.96566535610606), (13.922998986650642, 50.96566542432684), (13.922999050455463, 50.9656654991776), (13.922999106598636, 50.96566557993424), (13.922999154537052, 50.96566566581555), (13.922999193806968, 50.965665755990734), (13.922999224028498, 50.96566584958748), (13.92299924490929, 50.965665945700344), (13.922999256247348, 50.96566604339957), (13.922999257932993, 50.96566614174005), (13.922999249949916, 50.965666239770464), (13.922999232375346, 50.96566633654249), (13.922999205379293, 50.96566643111999), (13.922999169222907, 50.96566652258805), (13.922999124255954, 50.965666610061845), (13.92299900814523, 50.965666810970866), (13.922998623

In [12]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
folium.GeoJson(active_cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m